In [52]:
import numpy as np
import scipy
import struct
import os
from matplotlib.pylab import plt
from sklearn.neighbors import KNeighborsClassifier

In [53]:
path = "./mnist_data/"
def read(dataset = "training", datatype='images'):
    """
    Python function for importing the MNIST data set. It returns an iterator
    of 2-tuples with the first element being the label and the second element
    being a numpy.uint8 2D array of pixel data for the given image.
    """
    if dataset is "training":
        fname_img = os.path.join(path, 'train-images-idx3-ubyte')
        fname_lbl = os.path.join(path, 'train-labels-idx1-ubyte')
    elif dataset is "testing":
        fname_img = os.path.join(path, 't10k-images-idx3-ubyte')
        fname_lbl = os.path.join(path, 't10k-labels-idx1-ubyte')
        
    # Load everything in some numpy arrays
    with open(fname_lbl, 'rb') as flbl:
        magic, num = struct.unpack(">II", flbl.read(8))
        lbl = np.fromfile(flbl, dtype=np.int8)
    with open(fname_img, 'rb') as fimg:
        magic, num, rows, cols = struct.unpack(">IIII", fimg.read(16))
        img = np.fromfile(fimg, dtype=np.uint8).reshape(len(lbl), rows, cols)
    if(datatype=='images'):
        get_data = lambda idx: img[idx]
    elif(datatype=='labels'):
        get_data = lambda idx: lbl[idx]
    # Create an iterator which returns each image in turn
    for i in range(len(lbl)):
        yield get_data(i)
        
trainData=np.array(list(read('training','images')))
trainLabels=np.array(list(read('training','labels')))
testData=np.array(list(read('testing','images')))
testLabels=np.array(list(read('testing','labels')))

# Make data 2D
trainData = trainData.reshape(trainData.shape[0], -1)
testData = testData.reshape(testData.shape[0], -1)
# Make labels 2D
#trainLabels = trainLabels.reshape(trainLabels.shape[0], -1)
#testLabels = testLabels.reshape(testLabels.shape[0], -1)

In [54]:
#Base and mixin classes for instance reduction techniques

# Author: Dayvid Victor <dvro@cin.ufpe.br>
# License: BSD Style
import warnings
from abc import ABCMeta, abstractmethod

from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.neighbors.classification import KNeighborsClassifier

from sklearn.utils import check_array
from sklearn.externals import six


class InstanceReductionWarning(UserWarning):
    pass

# Make sure that NeighborsWarning are displayed more than once
warnings.simplefilter("always", InstanceReductionWarning)


class InstanceReductionBase(six.with_metaclass(ABCMeta, BaseEstimator)):

    """Base class for instance reduction estimators."""

    @abstractmethod
    def __init__(self):
        pass


class InstanceReductionMixin(InstanceReductionBase, ClassifierMixin):

    """Mixin class for all instance reduction techniques"""


    def set_classifier(self):
        """Sets the classified to be used in the instance reduction process
            and classification.

        Parameters
        ----------
        classifier : classifier, following the KNeighborsClassifier style
            (default = KNN)

        y : array-like, shape = [n_samples]
            Labels for X.

        Returns
        -------
        P : array-like, shape = [indeterminated, n_features]
            Resulting training set.

        q : array-like, shape = [indertaminated]
            Labels for P
        """

        self.classifier = classifier


    def reduce_data(self, X, y):
        """Perform the instance reduction procedure on the given training data.

        Parameters
        ----------
        X : array-like, shape = [n_samples, n_features]
            Training set.0

        y : array-like, shape = [n_samples]
            Labels for X.

        Returns
        -------
        X_ : array-like, shape = [indeterminated, n_features]
            Resulting training set.

        y_ : array-like, shape = [indertaminated]
            Labels for X_
        """
        pass

    def fit(self, X, y, reduce_data=True):
        """
        Fit the InstanceReduction model according to the given training data.

        Parameters
        ----------
        X : {array-like, sparse matrix}, shape = [n_samples, n_features]
            Training vector, where n_samples in the number of samples and
            n_features is the number of features.
            Note that centroid shrinking cannot be used with sparse matrices.
        y : array, shape = [n_samples]
            Target values (integers)
        reduce_data : bool, flag indicating if the reduction would be performed
        """
        self.X = X
        self.y = y

        if reduce_data:
            self.reduce_data(X, y)

        return self

    def predict(self, X, n_neighbors=1):
        """Perform classification on an array of test vectors X.

        The predicted class C for each sample in X is returned.

        Parameters
        ----------
        X : array-like, shape = [n_samples, n_features]

        Returns
        -------
        C : array, shape = [n_samples]

        Notes
        -----
        The default prediction is using KNeighborsClassifier, if the
        instance reducition algorithm is to be performed with another
        classifier, it should be explicited overwritten and explained
        in the documentation.
        """
        X = check_array(X)
        if not hasattr(self, "X_") or self.X_ is None:
            raise AttributeError("Model has not been trained yet.")

        if not hasattr(self, "y_") or self.y_ is None:
            raise AttributeError("Model has not been trained yet.")

        if self.classifier == None:
            self.classifier = KNeighborsClassifier(n_neighbors=n_neighbors)

        self.classifier.fit(self.X_, self.y_)
        return self.classifier.predict(X)


    def predict_proba(self, X):
        """Return probability estimates for the test data X.
        after a given prototype selection algorithm.

        Parameters
        ----------
        X : array, shape = (n_samples, n_features)
            A 2-D array representing the test points.

        Returns
        -------
        p : array of shape = [n_samples, n_classes], or a list of n_outputs
        of such arrays if n_outputs > 1.
        The class probabilities of the input samples. Classes are ordered
        by lexicographic order.
        """
        self.classifier.fit(self.X_, self.y_)
        return self.classifier.predict_proba(X)


In [93]:
# -*- coding: utf-8 -*-
"""
Condensed-Nearest Neighbors
"""

# Author: Dayvid Victor <victor.dvro@gmail.com>
#
# License: BSD 3 clause

import numpy as np


from sklearn.utils.validation import check_X_y
from sklearn.neighbors.classification import KNeighborsClassifier

#from base import InstanceReductionMixin


class CNN(InstanceReductionMixin):
    """Condensed Nearest Neighbors.

    Each class is represented by a set of prototypes, with test samples
    classified to the class with the nearest prototype.
    The Condensed Nearest Neighbors removes the redundant instances,
    maintaining the samples in the decision boundaries.

    Parameters
    ----------
    n_neighbors : int, optional (default = 1)
        Number of neighbors to use by default for :meth:`k_neighbors` queries.

    Attributes
    ----------
    `prototypes_` : array-like, shape = [indeterminated, n_features]
        Selected prototypes.

    `labels_` : array-like, shape = [indeterminated]
        Labels of the selected prototypes.

    `reduction_` : float, percentual of reduction.

    Examples
    --------
    >>> from protopy.selection.cnn import CNN
    >>> import numpy as np
    >>> X = np.array([[-1, -1], [-2, -1], [-3, -2], [1, 1], [2, 1], [3, 2]])
    >>> y = np.array([1, 1, 1, 2, 2, 2])
    >>> cnn = CNN()
    >>> cnn.fit(X, y)
    CNN(n_neighbors=1)
    >>> print(cnn.predict([[-0.8, -1]]))
    [1]

    See also
    --------
    sklearn.neighbors.KNeighborsClassifier: nearest neighbors classifier

    Notes
    -----
    The Condensed Nearest Neighbor is one the first prototype selection
    technique in literature.

    References
    ----------
    P. E. Hart, The condensed nearest neighbor rule, IEEE Transactions on
    Information Theory 14 (1968) 515–516.

    """

    def __init__(self, n_neighbors=1, M=None):
        self.n_neighbors = n_neighbors
        self.classifier = None
        self.M = M

    def reduce_data(self, X, y):

        X, y = check_X_y(X, y, accept_sparse="csr")

        if self.classifier == None:
            self.classifier = KNeighborsClassifier(n_neighbors=self.n_neighbors)

        prots_s = []
        labels_s = []

        classes = np.unique(y)
        self.classes_ = classes

        # Adding one sample from each class
        for cur_class in classes:
            mask = y == cur_class
            insts = X[mask]
            prots_s = prots_s + [insts[np.random.randint(0, insts.shape[0])]]
            labels_s = labels_s + [cur_class]

        self.classifier.fit(np.asarray(prots_s), np.asarray(labels_s))
        
        for sample, label in zip(X, y):
            if self.M is not None and len(prots_s) == self.M:
                break
            if self.classifier.predict(sample.reshape(1, -1)) != [label]:
                prots_s = prots_s + [sample]
                labels_s = labels_s + [label]
                #assert(len(prots_s) == len(labels_s))
                self.classifier.fit(prots_s, labels_s)
        

        if self.M is not None and len(prots_s) < self.M:
            to_add = self.M - len(prots_s)
            idx = np.random.randint(0, X.shape[0], size=to_add)
            prots_s =  np.concatenate((np.asarray(prots_s), X[idx]), axis=0)
            labels_s = np.concatenate((np.asarray(labels_s), y[idx]), axis=0)
            self.classifier.fit(prots_s, labels_s)
            
        self.X_ = np.asarray(prots_s)
        self.y_ = np.asarray(labels_s)
        self.reduction_ = 1.0 - float(len(self.y_))/len(y)
        return self.X_, self.y_

In [94]:
print("number of samples:{}".format(trainData.shape[0]))

number of samples:60000


In [47]:
# Imbalance the dataset
classes = np.unique(trainLabels)
b = np.random.uniform(0.30,0.7, 5)
# print(a)
sample_subsets = []
target_subsets = []
for cur_class in range(5):
    mask = trainLabels == cur_class
    insts = trainData[mask]
    a = np.random.randint(0, insts.shape[0], (int)(insts.shape[0]*(b[cur_class])))
    sample_subsets.append(insts[a])
    target_subsets.append(trainLabels[mask][a])

for cur_class in range(5, 10):
    mask = trainLabels == cur_class
    insts = trainData[mask]
    sample_subsets.append(insts)
    target_subsets.append(trainLabels[mask])
    
# permutations
X_ = np.asarray(sample_subsets)
y_ = np.asarray(target_subsets)
X = X_[0]
y = [0] * len(X_[0])
for i in range(1, 10):
    X = np.concatenate((X, X_[i]), axis=0)
    y += [i] * len(X_[i])
    
y = np.asarray(y)

for _ in range(10):
    a = np.random.permutation(X.shape[0])
    X = X[a]
    y = y[a]
    
trainData = X
trainLabels = y
print("number of samples:{}".format(trainData.shape[0]))

number of samples:43604


In [59]:
cnn_new = CNN(n_neighbors=1)
cnn_new.fit(trainData, trainLabels)
print("Number of samples reduced to:{}".format(cnn_new.X_.shape))
cnn_new.score(testData, testLabels)

Number of samples reduced to:(4968, 784)


0.9312

In [50]:
def choose_random_samples(X, y, M):
    
    #indexes = np.random.randint(0, X.shape[0], size=M, seed=42)
    indexes = np.random.choice(X.shape[0], size=M, replace=False)
    prots_s = X[indexes]
    labels_s = y[indexes]
        
    return np.asarray(prots_s).reshape(-1, 784), np.asarray(labels_s)

In [51]:
acc = []
for _ in range(1):
    neigh = KNeighborsClassifier(n_neighbors=1)
    ProtoData, ProtoLabels = choose_random_samples(trainData, trainLabels, 3813)
    neigh.fit(ProtoData, ProtoLabels) 
    acc.append(neigh.score(testData, testLabels))

print("Number of prototype samples:{}".format(ProtoData.shape[0]))
print("Average accuracy:{}".format(sum(acc)/len(acc)))

Number of prototype samples:3813
Average accuracy:0.9229


In [60]:
cnn_new = CNN(n_neighbors=1, M=100)
cnn_new.fit(trainData, trainLabels)
print("Number of samples reduced to:{}".format(cnn_new.X_.shape))
cnn_new.score(testData, testLabels)

Number of samples reduced to:(100, 784)


0.7405

In [61]:
cnn_new = CNN(n_neighbors=1, M=500)
cnn_new.fit(trainData, trainLabels)
print("Number of samples reduced to:{}".format(cnn_new.X_.shape))
cnn_new.score(testData, testLabels)

Number of samples reduced to:(500, 784)


0.8513

In [62]:
cnn_new = CNN(n_neighbors=1, M=1000)
cnn_new.fit(trainData, trainLabels)
print("Number of samples reduced to:{}".format(cnn_new.X_.shape))
cnn_new.score(testData, testLabels)

Number of samples reduced to:(1000, 784)


0.8888

In [95]:
cnn_new = CNN(n_neighbors=1, M=5000)
cnn_new.fit(trainData, trainLabels)
print("Number of samples reduced to:{}".format(cnn_new.X_.shape))
cnn_new.score(testData, testLabels)

Number of samples reduced to:(5000, 784)


0.9302

In [96]:
cnn_new = CNN(n_neighbors=1, M=10000)
cnn_new.fit(trainData, trainLabels)
print("Number of samples reduced to:{}".format(cnn_new.X_.shape))
cnn_new.score(testData, testLabels)

Number of samples reduced to:(10000, 784)


0.9484